In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
from datasets import DatasetDict
from datasets import load_dataset

dataset = load_dataset("IlyaGusev/gazeta")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for IlyaGusev/gazeta contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/gazeta
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
})

In [17]:
train_subset = dataset['train'].select(range(15000))
test_subset = dataset['test'].select(range(1000))
validation_subset = dataset['validation'].select(range(1000))

# Combine them back into a DatasetDict if needed
subdataset = DatasetDict({
    'train': train_subset,
    'test': test_subset,
    'validation': validation_subset
})

In [18]:
# dataset['train']['text'][40]
def tokenize_fn(example):
    model_inputs = tokenizer(example['text'], padding=True, truncation=True)
    model_inputs["labels"] = tokenizer(example['summary'], padding=True, truncation=True)['input_ids']
    return model_inputs

# def preprocess(example):
    
# dataset.select()
inputs = subdataset.map(tokenize_fn, batched = True)
inputs = inputs.map(remove_columns=['title', 'date', 'url'])
# inputs = inputs.rename_column('summary', 'labels')

inputs.set_format(type='torch')

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
inputs

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [20]:
from transformers import Trainer
from transformers import DataCollatorWithPadding, DataCollatorForSeq2Seq

In [21]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, 
    model=model
)

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test-trainer", 
    num_train_epochs = 5,
    eval_strategy="steps",
    eval_steps=400,
    logging_strategy="steps",
    logging_steps=150, 
    per_device_train_batch_size = 6,
    per_device_eval_batch_size = 6,
    report_to = 'none',
)

In [24]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=inputs["train"],
    eval_dataset=inputs["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [25]:
trainer.train()
# batched_input = data_collator(inputs['train'])

Step,Training Loss,Validation Loss
400,1.209300,1.195479
800,1.195100,1.163383
1200,1.168000,1.145369
1600,1.145100,1.122717
2000,1.132800,1.118830
2400,1.116100,1.098092
2800,1.115000,1.100636
3200,1.106400,1.083011
3600,1.099200,1.089127
4000,1.098000,1.073758


TrainOutput(global_step=12500, training_loss=1.0756810385131836, metrics={'train_runtime': 4029.6188, 'train_samples_per_second': 18.612, 'train_steps_per_second': 3.102, 'total_flos': 1.01506351104e+16, 'train_loss': 1.0756810385131836, 'epoch': 5.0})

In [ ]:
# pip install --upgrade transformers accelerate


In [31]:
from transformers import pipeline

In [34]:
pipe = pipeline("summarization", model=model, tokenizer=tokenizer)

In [36]:
my_inputs = "Один раз школьник Ваня отправился в поход в горы с классом. Они выехали рано утром, и дорога заняла несколько часов. Когда автобус остановился, перед ними открылась величественная картина горных вершин. Ребята сразу же начали подниматься по тропинке, наслаждаясь свежим воздухом и красивыми видами. Ваня был в восторге от природы вокруг: цветы, деревья и маленькие ручейки создавали сказочную атмосферОни шли долго, иногда останавливаясь на привалы, чтобы отдохнуть и перекусить. Учитель рассказывал интересные факты о горах, и Ваня узнал много нового. В какой-то момент они добрались до вершины, откуда открывался потрясающий вид на долину. Ваня чувствовал себя на вершине мира и радовался, что смог покорить эту высоту. На обратном пути все делились впечатлениями и мечтали о новых походах. Этот день навсегда остался в памяти Вани как один из самых ярких и увлекательных."

In [37]:
pipe(my_inputs)

Token indices sequence length is longer than the specified maximum sequence length for this model (650 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'ан оод в ор с классом отравилс в восторе от рирод вокру и красивми видами. ета срау авил, то отдонут о новоо на долном ролем, котор наватс активнми, а оедественно картина верин, расскавали итат удут дороами и открсаими'}]

In [45]:
UNK = tokenizer.unk_token_id
tokenized = tokenizer(my_inputs, truncation=True)['input_ids']

def count_unk(seq):
    n_unk = 0
    for el in seq:
        if el == UNK:
            n_unk+=1
    return n_unk
count_unk(tokenized)

112

## Итог
Слишком много русскоязычных текстов распознаются токенизатором как неизвестные, значит нужно либо использовать другую модель, либо другой токенизатор (но другой токенизатор потребует новый претрейн). Пока так(

In [50]:
# dataset = load_dataset("cnn_dailymail", "3.0.0")
# train_dataset = dataset['train']
# eval_dataset = dataset['validation']

# dataset['train']['article'][2]

# def check_for_none(example):
#     return {
#         "text_has_none": example['text'] is None,
#         "summary_has_none": example['summary'] is None,
#         "input_ids_has_none": example['input_ids'] is None,
#         "attention_mask_has_none": example['attention_mask'] is None
#     }

# none_check = inputs['train'].map(check_for_none)

# # Суммирование None значений по всем записям
# none_summary = {
#     "text": sum(none_check['text_has_none']),
#     "summary": sum(none_check['summary_has_none']),
#     "input_ids": sum(none_check['input_ids_has_none']),
#     "attention_mask": sum(none_check['attention_mask_has_none'])
# }

# print(none_summary)